In [16]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Collections.ObjectModel;
using Newtonsoft.Json;

Installed Packages Microsoft.Azure.Cosmos, 3.50.0 Newtonsoft.Json, 13.0.3

In [17]:
var endpoint = "https://localhost:8081";
var key ="C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(endpoint, key);
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [18]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

You need to enable the Indexing Metrics by using the QueryRequestOptions.
It is a great parameter to debug performance problems.
Don't keep it enabled in prod if you are not having issues.
It returns a JSON document which is not really easy to read.

In [27]:
public class SingleIndex{
    public string IndexSpec { get; set; }    
}

public class CompositeIndexes{
    public string IndexSpec { get; set; }
}

public class UtilizedIndexes{    
    public List<SingleIndex> SingleIndexes { get; set; }
    public List<CompositeIndexes> CompositeIndexes { get; set; }
}

public class PotentialIndexes{    
    public List<SingleIndex> SingleIndexes { get; set; }
    public List<CompositeIndexes> CompositeIndexes { get; set; }
}

public class IndexMetrics{
    public UtilizedIndexes UtilizedIndexes { get; set; }
    public PotentialIndexes PotentialIndexes { get; set; }
}



In [29]:
var cmd = "SELECT c.PostId FROM c WHERE c.OwnerUserId = 1 AND c.PostType= 'Question'";
var postQuery = new QueryDefinition(cmd);
var iterator = postContainer.GetItemQueryIterator<Post>(postQuery, requestOptions: new QueryRequestOptions
        {
            PopulateIndexMetrics = true
        });
var results = new List<Post>();

while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();    
    results.AddRange(response);
    response.RequestCharge.Display();
    var iMetrics = JsonConvert.DeserializeObject<IndexMetrics>(response.IndexMetrics);
    iMetrics.Display("application/json");
    //response.IndexMetrics.Display();
}

3.01